# Ingest GitHub Issues into Qdrant

This sample shows how to get started loading and querying dotnet/runtime repo GitHub issue embeddings using Azure OpenAI and the Qdrant SDK

## Install packages

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Qdrant.Client, 1.6.0-alpha.1"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 Qdrant.Client, 1.6.0-alpha.1

In [2]:
#r "nuget: Octokit, 9.0.0"
#r "nuget: Octokit.Reactive, 9.0.0"

Installed Packages Octokit, 9.0.0 Octokit.Reactive, 9.0.0

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.24129.1"

## Add using statements

In [4]:
using Azure;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;
using Octokit;

## Configure Azure OpenAI credentials

In [6]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");
var embeddingDeployment = await Kernel.GetInputAsync("Provide embedding name");

## Configure GitHub credentials 

You will need access token with rights to query and update issues.

In [38]:
var githubKey = await Kernel.GetPasswordAsync("Provide your Github api key");
var repoName = await Kernel.GetInputAsync("Provide repo");
var org = await Kernel.GetInputAsync("Provide org");

## Configure OpenAI client

In [39]:
OpenAIClient openAIClient = new (new System.Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

## Configure GitHub client

In [40]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

if (!string.IsNullOrEmpty(githubKey.GetClearTextPassword())) {
    Console.WriteLine("Using GitHub API token");
    var tokenAuth = new Credentials(githubKey.GetClearTextPassword());
    gitHubClient.Credentials = tokenAuth;
} else {
    Console.WriteLine("Using anonymous GitHub API");
}

Using GitHub API token


## Download data from GitHub

### Get labels from the repository

In [41]:
var allLabels = await gitHubClient.Issue.Labels.GetAllForRepository(org, repoName);

In [42]:
var areaLabels = allLabels.Where(label => label.Name.StartsWith("area-", StringComparison.OrdinalIgnoreCase)).ToList();

### Get all issues from the repository

In [43]:
var allIssues = new List<Issue>();

In [44]:
foreach(var label in areaLabels)
{
    var request = new RepositoryIssueRequest
    {
        Filter = IssueFilter.All
    };
    
    request.Labels.Add(label.Name);

    var apiOptions = new ApiOptions
    {
        PageSize = 50,
        PageCount = 1
    };

    var issues = await gitHubClient.Issue.GetAllForRepository(org, repoName, request, apiOptions);
    
    allIssues.AddRange(issues);
}

In [45]:
allIssues.Count()

4138

In [46]:
public record GitHubIssue(string Title, string Text, string Area, string Url);

In [66]:
var dataCollection = 
    allIssues
        .DistinctBy(issue => issue.Id)
        .Select(issue => 
            new GitHubIssue(
                issue.Title,
                issue.Body,
                issue.Labels?.Where(l => 
                    l.Name.StartsWith("area-",StringComparison.OrdinalIgnoreCase))
                        .FirstOrDefault()?
                        .Name?
                        .Replace("area-",string.Empty)?
                        .Replace("-"," "),
                issue.HtmlUrl));

## Helper functions to save and load to disk

In [48]:
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;

public async Task SaveIssuesToFileAsync(IEnumerable<GitHubIssue> data, string fileName)
{ 
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var issuesJson = JsonSerializer.Serialize(data,new JsonSerializerOptions(JsonSerializerOptions.Default){WriteIndented=true});
    await File.WriteAllTextAsync(filePath, issuesJson);
}

public async Task<GitHubIssue[]> LoadIssuesFromFileAsync(string fileName)
{
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var text = await File.ReadAllTextAsync(filePath);
    return JsonSerializer.Deserialize<GitHubIssue[]>(text);
}

In [67]:
await SaveIssuesToFileAsync(dataCollection, "issues.json");

## Naive Search

In [68]:
var dataCollection = await LoadIssuesFromFileAsync("issues.json");

In [69]:
dataCollection.Count()

4127

In [70]:
public string[] NaiveSearch(string query, IEnumerable<GitHubIssue> data,int resultLimit = 1)
{
    return data
            .Where(d => d.Text?.Contains(query)==true)
            .Select(d => d.Text)
            .Take(resultLimit)
            .ToArray();
}

In [71]:
NaiveSearch("What are the latest issues for AOT",dataCollection).Display();

[ ]

## Chunk issues

In [72]:
var dataCollection = await LoadIssuesFromFileAsync("issues.json");

In [73]:
dataCollection.Count()

4127

### Initialize collection of issues with chunks

In [74]:
public record TextWithEmbedding(string Text, float[] Embedding);
public record IssueWithChunks(GitHubIssue Issue,List<TextWithEmbedding> Chunks);

In [75]:
var issuesWithChunksCollection = 
    dataCollection
        .Select(issue => new IssueWithChunks(issue, new ()))
        .ToArray();

### Helper functions to save and load chunks to disk

In [76]:
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;

public async Task SaveIssuesWithChunksToFileAsync(IEnumerable<IssueWithChunks> data, string fileName)
{ 
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var issuesJson = JsonSerializer.Serialize(data,new JsonSerializerOptions(JsonSerializerOptions.Default){WriteIndented=true});
    await File.WriteAllTextAsync(filePath, issuesJson);
}

public async Task<IssueWithChunks[]> LoadIssuesWithChunksFromFileAsync(string fileName)
{
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var text = await File.ReadAllTextAsync(filePath);
    return JsonSerializer.Deserialize<IssueWithChunks[]>(text);
}

### Chunk data and generate embeddings

In [77]:
var tokenizer = await Tokenizer.CreateAsync(TokenizerModel.ada2);

var counter = 0;

foreach(var item in issuesWithChunksCollection.Take(100))
{
    var fullText = item.Issue.Text;
    if(string.IsNullOrWhiteSpace(fullText))
        continue;

    var chunks = 
        tokenizer
            .ChunkByTokenCountWithOverlap(fullText, 3000, 50)
            .Select(t => 
$"""
Title: {item.Issue.Title}
Area: {item.Issue.Area}

{t}
""")
            .Chunk(16)
            .ToArray();

    foreach(var chunk in chunks)
    {
        
        var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,chunk));
        item.Chunks.AddRange(
            embeddingResponse.Value.Data.Select(d => 
                new TextWithEmbedding(chunk[d.Index],d.Embedding.ToArray())));
    }

    if(counter % 50 == 0)
        await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");
    counter++;
}

await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");

In [78]:
issuesWithChunksCollection.Take(5).DisplayTable();

Issue Chunks GitHubIssue { Title = DependencyContext RuntimeGraph expected behavior., Text = To my knowledge this is not a defect but I am finding a discrepancy in the behavior of DependencyContext's [RuntimeGraph](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.dependencymodel.dependencycontex... Title DependencyContext RuntimeGraph expected behavior. Text To my knowledge this is not a defect but I am finding a discrepancy in the behavior of DependencyContext's [RuntimeGraph](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.dependencymodel.dependencycontext.runtimegraph?view=dotnet-plat-ext-7.0) property.

I've actually asked Bing's AI twice and received two different answers on this, but where does the information for RuntimeGraph come from? [Official documentation](https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.dependencymodel.dependencycontext?view=dotnet-plat-ext-7.0) says "ApplicationName.deps.json" but AI also said ApplicationName.runtimeconfig.json. However, it can't strictly be this as I've created an application that simply prints the RuntimeGraph to console and it differs depending on which platform I run it on. 

The application I've created uses default csproj options for a dotnet 6.0 Exe. I am compiling it in WSL on an Oracle Linux 8.7 system as RuntimeGetter and launching it against RuntimeGetter.dll. I do not set the RuntimeIdentifier.

The code is simple, it loads the Assembly using `LoadFromAssemblyPath` using arg[0] and then does:
```
 string retVal = "";

 foreach (var item in DependencyCtx.RuntimeGraph)
 {
 retVal += "Runtime: " + item.Runtime;
 retVal += " | Fallbacks: ";
 foreach (var item2 in item.Fallbacks)
 {
 retVal += item2 + " ";
 }
 }

 return retVal;
```

When run on Oracle Linux 8.7 I see the following:

> Runtime: ol.8.7-x64 | Fallbacks: ol.8.7 ol.8.6-x64 rhel.8.7-x64 ol.8.6 rhel.8.7 ol.8.5-x64 rhel.8.6-x64 ol.8.5 rhel.8.6 ol.8.4-x64 rhel.8.5-x64 ol.8.4 rhel.8.5 ol.8.3-x64 rhel.8.4-x64 ol.8.3 rhel.8.4 ol.8.2-x64 rhel.8.3-x64 ol.8.2 rhel.8.3 ol.8.1-x64 rhel.8.2-x64 ol.8.1 rhel.8.2 ol.8.0-x64 rhel.8.1-x64 ol.8.0 rhel.8.1 ol.8-x64 rhel.8.0-x64 ol.8 rhel.8.0 ol-x64 rhel.8-x64 ol rhel.8 rhel-x64 rhel linux-x64 linux unix-x64 unix any base

To me, this makes sense as the application was published from an OL8.7 system and is running on an OL8.7 system. I then copy it to an Amazon Linux 2 system and get the following:

> Runtime: alpine-x64 | Fallbacks: alpine linux-musl-x64 linux-musl linux-x64 linux unix-x64 unix any base Runtime: alpine.3.10-x64 | Fallbacks: alpine.3.10 alpine.3.9-x64 alpine.3.9 alpine.3.8-x64 alpine.3.8 alpine.3.7-x64 alpine.3.7 alpine.3.6-x64 alpine.3.6 alpine-x64 alpine linux-musl-x64 linux-musl linux-x64 linux unix-x64 unix any base Runtime: alpine.3.11-x64 | Fallbacks: alpine.3.11 alpine.3.10-x64 alpine.3.10 alpine.3.9-x64 alpine.3.9 alpine.3.8-x64 alpine.3.8 alpine.3.7-x64 alpine.3.7 alpine.3.6-x64 alpine.3.6 alpine-x64 alpine linux-musl-x64 linux-musl linux-x64 linux unix-x64 unix any base Runtime: alpine.3.12-x64 | Fallbacks: alpine.3.12 alpine.3.11-x64 alpine.3.11 alpine.3.10-x64 alpine.3.10 alpine.3.9-x64 alpine.3.9 alpine.3.8-x64 alpine.3.8 alpine.3.7-x64 alpine.3.7 alpine.3.6-x64 alpine.3.6 alpine-x64 alpine linux-musl-x64 linux-musl linux-x64 linux unix-x64 unix any base Runtime: alpine.3.13-x64 | Fallbacks: alpine.3.13 alpine.3.12-x64 alpine.3.12 alpine.3.11-x64 alpine.3.11 alpine.3.10-x64 alpine.3.10 alpine.3.9-x64 alpine.3.9 alpine.3.8-x64 alpine.3.8 alpine.3.7-x64 alpine.3.7 alpine.3.6-x64 alpine.3.6 alpine-x64 alpine linux-musl-x64 linux-musl linux-x64 linux unix-x64 unix any base Runtime: alpine.3.14-x64 | Fallbacks: alpine.3.14 alpine.3.13-x64 alpine.3.13 alpine.3.12-x64 alpine.3.12 alpine.3.11-x64 alpine.3.11 alpine.3.10-x64 alpine.3.10 alpine.3.9-x64 alpine.3.9 alpine.3.8-x64 alpine.3.8 alpine.3.7-x64 alpine.3.7 alpine.3.6-x64 alpine.3.6 alpine-x64 alpine linux-musl-x64 linux-mus

In [79]:
await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");

## Embedding Search

In [80]:
#r "nuget: System.Numerics.Tensors, 8.0.0-rc.2.23479.6"

Installed Packages System.Numerics.Tensors, 8.0.0-rc.2.23479.6

In [81]:
using System.Numerics.Tensors;

In [82]:
public class SimilarityComparer : ISimilarityComparer<float[]>
{
    public float Score(float[] a, float[] b)
    {
        return TensorPrimitives.CosineSimilarity(a,b);
    }
}

In [84]:
public async Task<string[]> EmbeddingSearchAsync(string query, IEnumerable<IssueWithChunks> data,int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var searchResults = 
        data
        .SelectMany(d => d.Chunks)
        .ScoreBySimilarityTo(embeddingVector,new SimilarityComparer(),c => c.Embedding)
        .OrderByDescending(e => e.Score)
        .Where(e => e.Score > 0.5)
        .Take(resultLimit)
        .Select(e => e.Value.Text)
        .ToArray();
    
    return searchResults;
}

In [85]:
(await EmbeddingSearchAsync("What are the latest issues for AOT", issuesWithChunksCollection, 3)).Display();

[ Title: [wasi] `System.Globalization.Tests` AOT build fails with `LLVM ERROR: out of memory` on Windows
Area: Codegen AOT mono

## Build Information
Build: https://dev.azure.com/dnceng-public/cbb18261-c48f-4abb-8651-8cdcb5474649/_build/results?buildId=481587
Build error leg or test failing: System.Globalization.Tests.WorkItemExecution
Pull request: https://github.com/dotnet/runtime/pull/95146
<!-- Error message template -->
## Error Message

Fill the error message using [step by step known issues guidance](https://github.com/dotnet/arcade/blob/main/Documentation/Projects/Build%20Analysis/KnownIssues.md#how-to-fill-out-a-known-issue-error-section).

<!-- Use ErrorMessage for String.Contains matches. Use ErrorPattern for regex matches (single line/no backtracking). Set BuildRetry to `true` to retry builds with this error. Set ExcludeConsoleLog to `true` to skip helix logs analysis. -->

```json
{
 "ErrorMessage": "",
 "ErrorPattern": "LLVM ERROR: out of memory.*C:.*ProxyProjectForAOTOnHelix.proj",
 "BuildRetry": false,
 "ExcludeConsoleLog": false
}
```

From the console log:
```
error : Failed to compile C:\helix\work\workitem\e\wasm_build\obj\wasm\for-build\aot-instances.dll.bc -> C:\helix\work\workitem\e\wasm_build\obj\wasm\for-build\aot-instances.dll.o [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : C:\helix\work\workitem\e\publish>C:\Windows\System32\chcp.com 65001 1>nul [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : C:\helix\work\workitem\e\publish>setlocal [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : C:\helix\work\workitem\e\publish>set errorlevel=dummy [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : C:\helix\work\workitem\e\publish>set errorlevel= [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : C:\helix\work\workitem\e\publish>C:\helix\work\correlation\build\\wasi-sdk\\bin\clang.exe "@C:\helix\work\correlation\build\microsoft.netcore.app.runtime.wasi-wasm\runtimes\wasi-wasm\native\src\wasi-default.rsp" "@C:\helix\work\workitem\e\wasm_build\obj\wasm\for-build\wasi-compile-bc.rsp" -c -o "C:\Users\ContainerAdministrator\AppData\Local\Temp\tmpmr5st5.tmp" "C:\helix\work\workitem\e\wasm_build\obj\wasm\for-build\aot-instances.dll.bc" [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : clang version 16.0.0 [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : Target: wasm32-unknown-wasi [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : Thread model: posix [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : InstalledDir: C:/helix/work/correlation/build//wasi-sdk//bin [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : (in-process) [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : "C:/helix/work/correlation/build/wasi-sdk/bin/clang.exe" -cc1 -triple wasm32-unknown-wasi -emit-obj -disable-free -clear-ast-before-backend -disable-llvm-verifier -discard-value-names -main-file-name aot-instances.dll.bc -mrelocation-model static -mframe-pointer=none -ffp-contract=on -fno-rounding-math -mconstructor-aliases -target-cpu generic -fvisibility=hidden -mllvm -treat-scalable-fixed-error-as-warning -debug-info-kind=constructor -dwarf-version=4 -debugger-tuning=gdb -v -fcoverage-compilation-dir=C:/helix/work/workitem/e/publish -resource-dir C:/helix/work/correlation/build/wasi-sdk/lib/clang/16 -Oz -fdebug-compilation-dir=C:/helix/work/workitem/e/publish -ferror-limit 19 -fgnuc-version=4.2.1 -vectorize-slp -o "C:\\Users\\ContainerAdministrator\\AppData\\Local\\Temp\\tmpmr5st5.tmp" -x ir "C:\\helix\\work\\workitem\\e\\wasm_build\\obj\\wasm\\for-build\\aot-instances.dll.bc" [C:\helix\work\workitem\e\publish\ProxyProjectForAOTOnHelix.proj]
error : clang -cc1 version 16.0.0 based upon LLVM 16.0.0 default target wasm32-wasi [C:\helix\work\workitem\e\publi

## Store in DB

### Start DB locally

In [86]:
docker run -d -p 6333:6333 -p 6334:6334 -v "$pwd/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

docker: error during connect: this error may indicate that the docker daemon is not running: Post "http://%2F%2F.%2Fpipe%2Fdocker_engine/v1.24/containers/create": open //./pipe/docker_engine: The system cannot find the file specified.
See 'docker run --help'.


Error: Command failed: SubmitCode: docker run -d -p 6333:6333 -p 6334:6334 -v "$pwd/q ...

In [40]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

### Initialize Qdrant client

In [41]:
var qdrantClient = new QdrantClient(host: "localhost",port: 6334,https:false);

### Create collection

In [42]:
var collectionName = "gh_issues";

In [43]:
var collections = await qdrantClient.ListCollectionsAsync();
if(collections.Where(x => x.Contains(collectionName)).Count() > 0) 
    await qdrantClient.DeleteCollectionAsync(collectionName);

In [44]:
await qdrantClient.CreateCollectionAsync(collectionName, new VectorParams { Size=1536, Distance=Distance.Cosine})

### Map issue embeddings to points

In [45]:
var vectors = 
    issuesWithChunksCollection
        .Where(d => d.Chunks.Count > 0)
        .SelectMany(d => 
            d.Chunks.Select(c => new {
                Embedding=c.Embedding,
                Text=$"<issueTitle>{d.Issue.Title}</issueTitle>\n<issueUrl>{d.Issue.Url}</issueUrl>\n<issueArea>{d.Issue.Area}</issueArea>\n<issueSnippet>{c.Text}</issueSnippet>"
                }))
        .ToList();
  

In [46]:
var points = vectors.Select(vector => 
{
    var point = new PointStruct
    {
        Id = new PointId { Uuid = Guid.NewGuid().ToString() },
        Vectors = vector.Embedding,
        Payload = 
            {
                ["text"] = vector.Text
            }
    };
    return point;
}).ToList();


### Insert data into Qdrant collection

In [47]:
await qdrantClient.UpsertAsync(collectionName,points);

## Get Count

In [48]:
await qdrantClient.CountAsync(collectionName)

95

## Search with Qdrant

In [49]:
public async Task<string[]> SearchWithQdrantAsync(string query, string collectionName, int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var results = await qdrantClient.SearchAsync(collectionName,embeddingVector, limit:(ulong)resultLimit);
    return results.Select(r => r.Payload["text"].StringValue).ToArray();
}

In [50]:
(await SearchWithQdrantAsync("What are the latest issues for AOT", collectionName, 3)).Display();

[ <issueTitle>[docs] Add Mono Library Mode and update Mono Profiled AOT</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/pull/94578</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [docs] Add Mono Library Mode and update Mono Profiled AOT
Area: Build mono

This PR looks to add/update documentation on [Mono's Library Mode](https://github.com/dotnet/runtime/issues/79377) and [Mono's Profiled AOT](https://github.com/dotnet/runtime/issues/69512) workflows.</issueSnippet>, <issueTitle>[browser][AOT] Space in path to published project on unix</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/pull/94306</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [browser][AOT] Space in path to published project on unix
Area: Build mono

Theoretically, enclosing included directories in quotations should be enough to escape spaces in Unix.

Reason: wbt use `artifacts/bin/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/9.0.0-alpha.1.23528.4/tools/bin/clang` that is `16.0.5`. Version `17.0.0` is already working fine on Unix, should not need any code edition, only unification of `wbt` -> `wbt artifacts` for all the platforms.
The issue reported clang `15.0.0` https://github.com/dotnet/runtime/issues/92335</issueSnippet>, <issueTitle>[wasm] WBT `SatelliteAssembliesTests.CheckThatSatelliteAssembliesAreNotAOTed` failing</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/issues/90458</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [wasm] WBT `SatelliteAssembliesTests.CheckThatSatelliteAssembliesAreNotAOTed` failing
Area: Build mono

Test failing with [rolling build](https://dev.azure.com/dnceng-public/public/_build/results?buildId=371479&view=results):

```
 [4/5] System.Private.CoreLib.dll.bc -> System.Private.CoreLib.dll.o 

WasmApp.Native.targets(379,5): error : Failed to compile /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -> /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.o 
WasmApp.Native.targets(379,5): error : emcc: warning: linker setting ignored during compilation: 'EXPORT_ES6' [-Wunused-command-line-argument] 
WasmApp.Native.targets(379,5): error : Killed
```

`WasmApp.Native.targets(379,5): error : emcc: error: '/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/8.0.0-rc.1.23411.1/tools/bin/clang++ -target wasm32-unknown-emscripten -fvisibility=default -mllvm -combiner-global-alias-analysis=false -mllvm -wasm-enable-sjlj -mllvm -disable-lsr -DEMSCRIPTEN --sysroot=/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Cache.linux-x64/8.0.0-rc.1.23411.1/tools/emscripten/cache/sysroot -Xclang -iwithsysroot/include/fakesdl -Xclang -iwithsysroot/include/compat -msimd128 -O0 -g3 -fwasm-exceptions -c /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -o /tmp/tmpeo8pzM.tmp' failed (returned 137) [took 128.818s] `

[Changes since last passing rolling build](https://github.com/dotnet/runtime/compare/64a67710163...55828b9aa91).

This includes commits from @pavelsavara @ivanpovazan @vargaz @radekdoulik 
<!-- Error message template -->
### Known Issue Error Message

Fill the error message using [step by step known issues guidance](https://github.com/dotnet/arcade/blob/main/Documentation/Projects/Build%20Analysis/KnownIssues.md#how-to-fill-out-a-known-issue-error-section).

<!-- Use ErrorMessage for String.Contains matches. Use ErrorPattern for regex matches (single line/no backtracking). Set BuildRetry to `true` to retry builds with this error. Set ExcludeConsoleLog to `true` to skip helix logs analysis. -->

```json
{
 "ErrorMessage": "error : Failed to compile ",
 "ErrorPattern": "",
 "BuildRetry": false,
 "ExcludeConsoleLog": false
}
```


<!-